In [ ]:
import cv2
import numpy as np


def white_img_extract(FILENAME):
    """전체 화면에서 흰 화면만 뽑아내기

    Args:
        FILENAME (_type_): 파일이름

    Returns:
        _type_: ndarray
    """
    img = cv2.imread(FILENAME)

    # 전처리 과정
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.Canny(blur, 50, 200)

    # 윤곽선 찾아내기
    contours, hierarchy = cv2.findContours(
        thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE
    )
    # contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # 이미지 크롭
    sl = []
    box_sl = []

    for cnt in contours:
        pt, hw, ang = cv2.minAreaRect(cnt)
        # minAreaRect 입력값: contours, 반환값: 좌측상단 점, (가로,세로), 각도
        if hw[0] > 1000 and hw[1] > 500:
            # if hw[0]>500:
            sl.append([pt, hw, ang])

    for i in sl:
        box = cv2.boxPoints((i[0], i[1], i[2]))
        box_sl.append(np.int0(box))

    # 이미지 보여주기
    for box in box_sl:
        img = cv2.drawContours(img, [box], 0, (0, 255, 0), 3)

    cv2.namedWindow("img", flags=cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
    cv2.waitKey()
    cv2.destroyAllWindows()

    # 원근변환 이용해서 필요한 이미지 추출
    box = np.float32(box)
    next_arr = np.array([[0, 0], [1200, 0], [1200, 1600], [0, 1600]], dtype=np.float32)
    per_mat = cv2.getPerspectiveTransform(box, next_arr)
    per = cv2.warpPerspective(img, per_mat, (1200, 1600))

    # 원근변환한 이미지 보여주기
    cv2.imshow("per", per)
    cv2.waitKey()
    cv2.destroyAllWindows()
    return per


# 예시
if "__main__" == __name__:
    white_img_extract("case2_2.jpg")